# Negative-Binomial Plots of Number of Secondary Cases

This notebook contains the code for plotting the distribution of number of secondary cases along with their Negative-Binomial fits. As such, the code still requires a bit of manual adaptation for each experiment. So the code here corresponds to the code to generate the plots in the paper.


In [1]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [2]:
import itertools
import numpy as np
from scipy import stats as sps
import scipy.optimize
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
from lib.calibrationSettings import (
    calibration_lockdown_dates, 
    calibration_states,
)
from lib.experiment import load_summary
from lib import plot as lib_plot
from lib import rt_nbinom  # Module for NB fit of Rt/kt

---

## Experiment in continued lockdown

In [94]:
def plot_daily_rt(country, area, num_weeks, plot_rt=True, plot_pmf=True):
    print(f"{country}-{area}: {num_weeks}")
    print('-'*20)
    # Load the experiment summary
    summary_name = f'continued-lockdown-{country}-{area}-extended_lockdown_weeks={num_weeks}.pk'
    result = load_summary(summary_name)
    # Extract the lockdown start time for the country
    country_code = result.metadata.country
    lockdown = calibration_lockdown_dates[country_code]
    print((f"{country_code}: simulation starts on {result.metadata.start_date}, "
           f"lockdown starts on {lockdown['start']}"))
    # Plot daily Rt over time
    if plot_rt:
        print('-- Plot daily Rt over time...')
        plotter = lib_plot.Plotter()
        plotter.plot_daily_nbinom_rts(
            result=result,
            slider_size=24.0, window_size=24.*5, end_cutoff=24.*7,
            ymax=4.4,
            lockdown_label='Lockdown', 
            lockdown_at=pd.to_datetime(lockdown['start']), 
            lockdown_xshift=1.1 * result.summary.max_time / 1560.0,
            lockdown_label_y=2.7,
            xtick_interval=2,
            figsize=(2.65, 1.9),
            figformat='neurips-double',
            filename=f"{result.metadata.experiment_info}_{result.metadata.simulation_info}-daily-Rt"
    )
    # Plot 2ndary cases distributions
    if plot_pmf:
        print('-- Plot 2ndary cases distributions...')
        if num_weeks == 100:
            t0_range=[0.0 * 24,   # the week before lockdown
                      15.0 * 24,  # right after start of lockdown
                     ]
            window_size=10.0*24
            label_range=['pre-lockdown',
                         'in-lockdown'
                        ]
        else:
            t0_range=[0.0 * 24,   # the week before lockdown
                      11.0 * 24,  # right after start of lockdown
                      38.0 * 24   # right after end of lockdown
                     ]
            window_size=7.0*24
            label_range=['pre-lockdown',
                         'in-lockdown',
                         'post-lockdown'
                        ]
        plotter = lib_plot.Plotter()
        plotter.plot_nbinom_distributions(
            result=result,
            x_range=np.arange(0, 20),
            t0_range=t0_range,
            window_size=window_size,
            label_range=label_range,
            ymax=0.85,
            figsize=lib_plot.FIG_SIZE_NEURIPS_TRIPLE_TALL,
            figformat='neurips-double',
            filename=f"{result.metadata.experiment_info}_{result.metadata.simulation_info}-pmf"
        )
    print()


places = [
    ('CH', 'BE'), 
    ('CH', 'JU'), 
    ('CH', 'TI'),
    ('GER', 'KL'),  
#     ('GER', 'RH'), 
    ('GER', 'TU')
]

for country, area in places:
    for num_weeks in [4, 100]:
        plot_daily_rt(country, area, num_weeks, plot_rt=False, plot_pmf=True)


# Crop pdf margins
!bash crop_pdfs.sh plots/daily-nbinom-rts-*.pdf
!bash crop_pdfs.sh plots/prob-secondaryCases-*.pdf

GER-RH: 4
--------------------
GER: simulation starts on 2020-03-10, lockdown starts on 2020-03-23
-- Plot 2ndary cases distributions...
Save: plots/prob-secondaryCases-continued-lockdown-GER-RH_extended_lockdown_weeks=4-pmf-0-pre-lockdown.pdf
Save: plots/prob-secondaryCases-continued-lockdown-GER-RH_extended_lockdown_weeks=4-pmf-1-in-lockdown.pdf
Save: plots/prob-secondaryCases-continued-lockdown-GER-RH_extended_lockdown_weeks=4-pmf-2-post-lockdown.pdf

GER-RH: 100
--------------------
GER: simulation starts on 2020-03-10, lockdown starts on 2020-03-23
-- Plot 2ndary cases distributions...
Save: plots/prob-secondaryCases-continued-lockdown-GER-RH_extended_lockdown_weeks=100-pmf-0-pre-lockdown.pdf
Save: plots/prob-secondaryCases-continued-lockdown-GER-RH_extended_lockdown_weeks=100-pmf-1-in-lockdown.pdf

PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko Oberdiek.
==> 1 page written on `tmp.pdf'.
PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko Oberdiek.
==> 1 page 

---

Custom plot parameter values for GER-RH

In [95]:
summary_name = 'continued-lockdown-GER-RH-extended_lockdown_weeks=4.pk'

# Load the experiment summary
result = load_summary(summary_name)

# Extract the lockdown start time for the country
country_code = result.metadata.country
lockdown = calibration_lockdown_dates[country_code]
print((f"{country_code}: simulation starts on {result.metadata.start_date}, "
       f"lockdown starts on {lockdown['start']}"))

plotter = lib_plot.Plotter()

GER: simulation starts on 2020-03-10, lockdown starts on 2020-03-23


In [96]:
t0_range = [
    1.0 * 24,   # the week before lockdown
    15.0 * 24,  # right after start of lockdown
    38.0 * 24   # right after end of lockdown
]
window_size = 10.0*24
label_range = [
    'pre-lockdown',
    'in-lockdown',
    'post-lockdown'
]
x_range = np.arange(0, 20)

In [97]:
interval_range = [(t0, t0 + window_size) for t0 in t0_range]
df = plotter._compute_nbinom_distributions(result=result, x_range=x_range, interval_range=interval_range)

In [98]:
plotter = lib_plot.Plotter()
plotter.plot_nbinom_distributions(
    result=result,
    df=df,
    x_range=x_range,
    t0_range=t0_range,
    window_size=window_size,
    label_range=label_range,
    ymax=0.85,
    figsize=lib_plot.FIG_SIZE_NEURIPS_TRIPLE_TALL,
    figformat='neurips-double',
    filename=f"{result.metadata.experiment_info}_{result.metadata.simulation_info}-pmf"
)

# Crop pdf margins
!bash crop_pdfs.sh plots/prob-secondaryCases-*.pdf

Save: plots/prob-secondaryCases-continued-lockdown-GER-RH_extended_lockdown_weeks=4-pmf-0-pre-lockdown.pdf
Save: plots/prob-secondaryCases-continued-lockdown-GER-RH_extended_lockdown_weeks=4-pmf-1-in-lockdown.pdf
Save: plots/prob-secondaryCases-continued-lockdown-GER-RH_extended_lockdown_weeks=4-pmf-2-post-lockdown.pdf
PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko Oberdiek.
==> 1 page written on `tmp.pdf'.
PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko Oberdiek.
==> 1 page written on `tmp.pdf'.
PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko Oberdiek.
==> 1 page written on `tmp.pdf'.
PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko Oberdiek.
==> 1 page written on `tmp.pdf'.
PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko Oberdiek.
==> 1 page written on `tmp.pdf'.
PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko Oberdiek.
==> 1 page written on `tmp.pdf'.
PDFCROP 1.38, 2012/11/02 - Copyright (c) 2002-2012 by Heiko O